<h1><b>This notebook is intended to plot markers' histogram for a given mass cytometry sample within a given batch</b></h1>

In [1]:
source("../UsedSourceCodes/mass_cytometry_functions.R")
library(caret)
library(glue)
library(emdist)
library(egg)

ERROR: Error in library(i, character.only = TRUE): there is no package called 'MASS'


In [ ]:
options(repr.plot.width=12, repr.plot.height=10)

In [ ]:
needed_antibody<- c('141Pr_CD196','142Nd_CD19', '143Nd_CD5',  '144Nd_CD38', '146Nd_IgD','147Sm_CD11c', '148Nd_CD16',
                    '149Sm_CCR4', '150Nd_CD43', '151Eu_CD69', '152Sm_CD21', '153Eu_CXCR5', '154Sm_CD62L', '158Gd_CD27',
                     '159Tb_CD22', '160Gd_CD14', '163Dy_CXCR3', '164Dy_CD23', '166Er_CD24', '167Er_CCR7',  '171Yb_CD20',
                    '172Yb_IgM', '173Yb_HLA-DR', '174Yb_CD49d', '175Lu_CXCR4', '176Yb_CD56')
needed_antibody1<- c('141Pr_CD196','142Nd_CD19', '143Nd_CD5',  '144Nd_CD38', '146Nd_IgD','147Sm_CD11c',
                   '150Nd_CD43', '151Eu_CD69', '152Sm_CD21', '153Eu_CXCR5', '154Sm_CD62L', '158Gd_CD27',
                     '159Tb_CD22', '163Dy_CXCR3', '164Dy_CD23', '166Er_CD24', '167Er_CCR7',  '171Yb_CD20',
                    '172Yb_IgM', '173Yb_HLA-DR', '174Yb_CD49d', '175Lu_CXCR4')

#####################
<h1><b>calculating EMD between fcs files for every marker</b></h1>

####################

In [ ]:
emd_for_each_marker_for_two_fcs<- function(path_needed1, path_needed2, transform, sampling_number_emd, needed_antibody){
        dataframe_res1<- fcs_to_dataframe(path_needed1, 
                        needed_antibody,
                        transform)
        dataframe_res2<- fcs_to_dataframe(path_needed2, 
                        needed_antibody,
                        transform)
        distances<- c()
        for (column_needed in c(1:dim(dataframe_res2)[2])){
            A<- sample(dataframe_res1[ ,column_needed], sampling_number_emd, replace = TRUE)
            A<- matrix(A)
            B<- sample(dataframe_res2[ ,column_needed], sampling_number_emd, replace = TRUE)
            B<- matrix(B)
            distance<- (suppressWarnings(emd2d(A,B,dist="euclidean"))/sampling_number_emd)*100
            distances<- append(distances, distance)
        }
        results<- data.frame(column_names = colnames(dataframe_res2), distance = distances)
        return(results)
}




emd_for_each_marker_for_two_dataframe<- function(dataframe_res1, dataframe_res2, sampling_number_emd, needed_antibody){

        distances<- c()
        for (column_needed in c(1:dim(dataframe_res2)[2])){
            A<- sample(dataframe_res1[ ,column_needed], sampling_number_emd, replace = TRUE)
            A<- matrix(A)
            B<- sample(dataframe_res2[ ,column_needed], sampling_number_emd, replace = TRUE)
            B<- matrix(B)
            distance<- (suppressWarnings(emd2d(A,B,dist="euclidean"))/sampling_number_emd)*100
            distances<- append(distances, distance)
        }
        results<- data.frame(column_names = colnames(dataframe_res2), distance = distances)
        return(results)
}

In [ ]:
histogram_of_each_column_for_three_dataframes<- function(dataframe1, dataframe2, dataframe3, title, xlab, bin, sampling_number, main_title){
   #' given dataframe
   #' given title name
   #' given xlab value
   #' give bin size
   #' output histogram of all columns
 dataframe1 <- dataframe1[sample(nrow(dataframe1), sampling_number, replace = TRUE), ]
 dataframe2<- dataframe2[sample(nrow(dataframe2), sampling_number, replace = TRUE), ]
 dataframe3<- dataframe3[sample(nrow(dataframe3), sampling_number, replace = TRUE), ]
 p<- ggplot() + 
    geom_density(data = gather(dataframe1), aes(value), color='green', fill= 'green', adjust = 1.5, alpha = 0.1, lwd= 1.5)+ 
    facet_wrap(~key, scales ='free', ncol = 3)+
    xlab(xlab)+
    geom_density(data = gather(dataframe2), aes(value), color='red', fill= 'red', adjust = 1.5, alpha = 0.1, lwd= 1.5)+
    facet_wrap(~key, scales = 'free', ncol = 3)+
    xlab(xlab)+
    geom_density(data = gather(dataframe3), aes(value), color='blue', fill= 'blue', adjust = 1.5, alpha = 0.1, lwd= 1.5)+
    facet_wrap(~key, scales = 'free', ncol = 3)+
    xlab(xlab)+
    ylab("Frequency")+
    ggplot2::theme(plot.title = element_text( size=16, face="bold.italic", hjust = 0.5),
                   axis.text = ggplot2::element_text(face="bold", colour="black", size=12))+
    ggtitle(main_title)
  return(p)  
}


histogram_of_each_column_for_two_dataframes<- function(df1, df2, title, xlab, bin, sampling_number, main_title){
   #' given dataframe
   #' given title name
   #' given xlab value
   #' give bin size
   #' output histogram of all columns
 dataframe1 <- df1[sample(nrow(df1), sampling_number, replace = TRUE), ]
 dataframe2<- df2[sample(nrow(df2), sampling_number, replace = TRUE), ]
 p<- ggplot() + 
    geom_density(data = gather(dataframe1), aes(value), color='green', fill= 'green', adjust = 1.5, alpha = 0.1, lwd= 1.5)+ 
    facet_wrap(~key, scales ='free', ncol = 3)+
    xlab(xlab)+
    geom_density(data = gather(dataframe2), aes(value), color='green', fill= 'blue', adjust = 1.5, alpha = 0.1, lwd= 1.5)+
    facet_wrap(~key, scales = 'free', ncol = 3)+
    xlab(xlab)+
    ylab("Frequency")+
    ggplot2::theme(plot.title = element_text( size=16, face="bold.italic", hjust = 0.5),
                   axis.text = ggplot2::element_text(face="bold", colour="black", size=12))+
    ggtitle(main_title)
  return(p)  
}


tag_facet <- function(p, open = "(", close = ")", tag_pool = letters, x = -Inf, y = Inf, 
                      hjust = -0.5, vjust = 1.5, fontface = 2, family = "", colour, ...) {
  
  gb <- ggplot_build(p)
  lay <- gb$layout$layout
  tags <- cbind(lay, label = paste0(open, tag_pool[lay$PANEL], close), x = x, y = y)
  p + 
    geom_text(data = tags, aes_string(x = "x", y = "y", label = "label"),colour=colour, ..., hjust = hjust, 
                vjust = vjust, fontface = fontface, family = family, inherit.aes = FALSE)+
    scale_colour_manual(values= colour)  
}

In [ ]:
####use above functions

In [ ]:
needed_antibody1<- c('141Pr_CD196','142Nd_CD19', '143Nd_CD5',  '144Nd_CD38', '146Nd_IgD','147Sm_CD11c',
                   '150Nd_CD43', '151Eu_CD69', '152Sm_CD21', '153Eu_CXCR5', '154Sm_CD62L', '158Gd_CD27',
                     '159Tb_CD22', '163Dy_CXCR3', '164Dy_CD23', '166Er_CD24', '167Er_CCR7',  '171Yb_CD20',
                    '172Yb_IgM', '173Yb_HLA-DR', '174Yb_CD49d', '175Lu_CXCR4')

In [ ]:
needed_antibody2<- c('141Pr_CD196','142Nd_CD19', '144Nd_CD38', 
                   '152Sm_CD21', '153Eu_CXCR5', '154Sm_CD62L')


In [ ]:
###parameters
numberofcellsusedforplot <- 1500
need <- needed_antibody2
batch1 <- "1st" 
batch2 <- "2nd"
samplenumber = "3620"
Dpath = "D:/AAAAAA_PhD_data/"

In [ ]:
#######

In [ ]:
path_needed1<- output_path(samplenumber, glue("{Dpath}Data/experiment_UMvsM/"), batch1)
path_needed2<- output_path(samplenumber, glue("{Dpath}Data/experiment_UMvsM/"), batch2)
#path_needed3<- "D:/AAAAAA_PhD_data/Data/all_cytonorm_normalized/Batch_6_19nonanchor1.fcs"


filetocode = read.csv(".././R3_ML_mut_vs_unmut_scripts/filenametocode.csv")
filetocode <- data.frame(filetocode)
rows_with_value <- filetocode[filetocode$name == samplenumber, ]
rows_with_value <- rows_with_value$code
#string_needed1 <- rows_with_value[grep(paste0("^Batch_", substr(batch1, 1, 1)), rows_with_value)]
string_needed2 <- rows_with_value[grep(paste0("^Batch_", substr(batch2, 1, 1)), rows_with_value)]

path_needed3 <- paste0(glue("{Dpath}Data/all_cytonorm_normalized/"), string_needed2)

In [ ]:
dataframe_res1<- fcs_to_dataframe(path_needed1, 
                need,
                FALSE)
dataframe_res2<- fcs_to_dataframe(path_needed2, 
                need,
                FALSE)
dataframe_res3<- fcs_to_dataframe(path_needed3, 
                need,
                FALSE)

In [ ]:
ans1<- emd_for_each_marker_for_two_fcs(path_needed1, path_needed2, TRUE, numberofcellsusedforplot, need)
ans2<- emd_for_each_marker_for_two_fcs(path_needed1, path_needed3, TRUE, numberofcellsusedforplot, need)

In [ ]:
sum(ans1$distance - ans2$distance)/sum(ans1$distance)*100

In [ ]:
main_title<- glue("Overlaid histograms of sampled cells for sample {samplenumber} from batch {substr(batch1, 1, 1)} (green), \n batch {substr(batch2, 1, 1)} before correction (red), and after correction (blue)\n by ML-based method\n\n")

In [ ]:
p<- histogram_of_each_column_for_three_dataframes(dataframe_res1, dataframe_res2, dataframe_res3, paste0(basename(path_needed) , " normalized"), "markers expression", 100, 2000, main_title) ###for scaled dataframe

In [ ]:
p<- tag_facet(p, 
              open = "emd ",
          tag_pool = round(ans1$distance, digit = 3),
             colour = "red",
              fontface = 4)
p<- tag_facet(p, x = Inf, y = Inf, 
          hjust = 1.5,
          open = "emd ",
          tag_pool = round(ans2$distance, digit = 3),
             colour = "blue",
              fontface = 4)

In [ ]:
p